<a href="https://colab.research.google.com/github/tituslhy/Face-mask-classification/blob/main/Create_VertexAI_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [10]:
from google.colab import auth
auth.authenticate_user()

project_id = 'mask-classification-352301'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://cloud-ai-platform-c3cb6662-d01b-4eba-946b-2b441a4eca6e/
gs://masketeers-mle-facemaskdata/
gs://masketeers-mle-facemasklogs/
gs://masketeers-mle-facemaskmodels/


In [33]:
# Imports the Google Cloud client library
import os
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = 'masketeers-mle-facemaskdata'
paths_in_bucket = ['Test/WithMask','Test/WithoutMask']

In [19]:
def write_csv(output_name,bucket_name,path_in_bucket,label):
  blobs = storage_client.list_blobs(bucket_name, prefix=path_in_bucket)
  filename = output_name+'.csv'
  with open(filename, 'w+') as f:
    for blob in blobs:
        if '.png' in blob.name:
            bucket_path = 'gs://' + os.path.join(bucket_name, blob.name)
            f.write(', '.join([bucket_path, label]))
            f.write("\n")
  print(f'{filename} write complete!')

In [22]:
filenames = ['facemask_test_with_mask','facemask_test_without_mask']
labels = ['WithMask','WithoutMask']

In [23]:
for i in range(len(filenames)):
  write_csv(filenames[i],bucket_name,paths_in_bucket[i],labels[i])

facemask_test_with_mask.csv write complete!
facemask_test_without_mask.csv write complete!


In [38]:
import pandas as pd

In [43]:
df1.columns

Index(['gs://masketeers-mle-facemaskdata/Test/WithMask/1163.png', 'label'], dtype='object')

In [44]:
df1 = pd.read_csv('facemask_test_with_mask.csv')
df1.rename(columns = {'gs://masketeers-mle-facemaskdata/Test/WithMask/1163.png':'Path',' WithMask':'label'},inplace=True)
df1.head()

,Path,label
0,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
1,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
2,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
3,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
4,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask


In [48]:
df2 = pd.read_csv('facemask_test_without_mask.csv')
df2.rename(columns = {'gs://masketeers-mle-facemaskdata/Test/WithoutMask/1.png':'Path',' WithoutMask':'label'},inplace=True)
df2.head()

,Path,label
0,gs://masketeers-mle-facemaskdata/Test/WithoutM...,WithoutMask
1,gs://masketeers-mle-facemaskdata/Test/WithoutM...,WithoutMask
2,gs://masketeers-mle-facemaskdata/Test/WithoutM...,WithoutMask
3,gs://masketeers-mle-facemaskdata/Test/WithoutM...,WithoutMask
4,gs://masketeers-mle-facemaskdata/Test/WithoutM...,WithoutMask


In [49]:
test = pd.concat([df1, df2], axis=0)
print(len(test)==len(df1)+len(df2))

True


In [51]:
test.head()

,Path,label
0,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
1,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
2,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
3,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask
4,gs://masketeers-mle-facemaskdata/Test/WithMask...,WithMask


In [52]:
filename = 'facemask_unit_test.csv'
test.to_csv(filename)
print('Task completed!')

Task completed!


In [53]:
# Uploading csv file to the bucket
bucket = storage_client.get_bucket(bucket_name)
path_name = 'Unit Test'
destination_blob_name = os.path.join(path_name, filename)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(filename)
print('Task completed!')

Task completed!
